# OpenMendel Tutorial on Iterative Hard Thresholding

### Last update: 10/16/2019

### Julia version

`MendelIHT.jl` currently supports Julia 1.0 and 1.2 on Mac, Linux, and Windows, but it currently an unregistered package. To install, press `]` to invoke the package manager mode and install these packages by typing:

```
add https://github.com/OpenMendel/SnpArrays.jl
add https://github.com/OpenMendel/MendelSearch.jl
add https://github.com/OpenMendel/MendelBase.jl
add https://github.com/OpenMendel/MendelIHT.jl
```

For this tutorial you will also need a few registered packages. Add them by typing:

```
add DataFrames, Distributions, BenchmarkTools, Random, LinearAlgebra, GLM
```

For reproducibility, the computer spec and Julia version is listed below.

In [1]:
versioninfo()

Julia Version 1.2.0
Commit c6da87ff4b (2019-08-20 00:03 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


## Documentation

This is a short tutorial. For detailed documentation, please visit our [home page](https://openmendel.github.io/MendelIHT.jl/latest/). 

### When to use Iterative Hard Thresholding

Continuous model selection is advantageous in situations where the multivariate nature of the regressors plays a significant role together. As an alternative to traditional SNP-by-SNP association testing, iterative hard-thresholing (IHT) performs continuous model selection on a GWAS dataset $\mathbf{X} \in \{0, 1, 2\}^{n \times p}$ and continuous phenotype vector $\mathbf{y}$ by maximizing the loglikelihood $L(\beta)$ subject to the constraint that $\beta$ is $k-$sparse. This method has the edge over LASSO because IHT does not shrink estimated effect sizes. Parallel computing is offered through `q-`fold cross validation.

### Appropriate Datasets and Example Inputs 

All genotype data **must** be stored in the [PLINK binary genotype format](https://www.cog-genomics.org/plink2/formats#bed) where at least the triplets `.bim`, `.bed` and `.fam` must all be present. Additional non-genetic covariates should be imported separately by the user. In the examples below, we first simulate phenotypes from the Normal, Bernoulli, Poisson, and Negative Binomial family, and then attempt to fit the corresponding model using our IHT implementation. We can examine reconstruction behavior as well as the ability for cross validation to find the true sparsity parameter.


### Missing Data

`MendelIHT` assumes there are no missing genotypes, since it uses linear algebra functions defined in [SnpArrays.jl](https://openmendel.github.io/SnpArrays.jl/latest/man/snparray/#linear-algebra-with-snparray). Therefore, you must first impute missing genotypes before you use MendelIHT. SnpArrays.jl offer basic quality control routines such as filtering, but otherwise, our own software [option 23 of Mendel](http://software.genetics.ucla.edu/download?package=1) is a reasonable choice. Open Mendel will soon provide a separate package `MendelImpute.jl` containing new imputation strategies such as alternating least squares.  

### Cross Validation and Regularization paths

We usually have very little information on how many SNPs are affecting the phenotype. In a typical GWAS study, anywhere between 1 to thousands of SNPs could play a role. Thus ideally, we can test many different models to find the best one. MendelIHT provides 2 ways for one to perform this automatically: user specified regulartization paths, and $q-$fold [cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics). Users should know that, in the first method, increasing the number of predictors will almost always decrease the error, but as a result introduce overfitting. Therefore, in most practical situations, it is highly recommended to combine this method with cross validation. In $q-$fold cross validation, samples are divided into $q$ disjoint subsets, and IHT fits a model on $q-1$ of those sets data, then computes the [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) tested on the $qth$ samples. Each $q$ subsets are served as the test set exactly once. This functionality of `MendelIHT.jl` natively supports parallel computing. 

# Example 1: Quantitative Traits

Quantitative traits are continuous phenotypes that can essentially take on any real number. In this example, we first simulate $y_i \sim x_i^T\beta + \epsilon_i$ where $\epsilon_i \sim N(0, 1)$ and $\beta_i \sim N(0, 1)$. Then using just the genotype matrix $X$ and phenotype vector $y$, we use IHT to recover the simulated $\beta$. 

In [2]:
#first add workers needed for parallel computing. Add only as many CPU cores you have 
using Distributed
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [3]:
#load necessary packages
using MendelIHT
using SnpArrays
using DataFrames
using Distributions
using BenchmarkTools
using Random
using LinearAlgebra
using GLM

### Step 1: Simulat data with k true predictors, from distribution d and with link l.

In [4]:
n = 1000
p = 10000
k = 10
d = Normal
l = canonicallink(d())

IdentityLink()

### Step 2: Construct snpmatrix, covariate files, and true model b

The SnpBitMatrix type (`xbm` below) is necessary for performing linear algebra directly on raw genotype files without expanding the matrix to numeric floating points. Here `undef` in the 3rd argument simply indicates that the matrix `x` will be stored in RAM. Please visit SnpArrays' [documentation](https://openmendel.github.io/SnpArrays.jl/latest/) for more detailed description. 

In [5]:
Random.seed!(1111) #set random seed
x = simulate_random_snparray(n, p, undef)
xbm = SnpBitMatrix{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true); 
z = ones(n, 1); # only nongenetic covariate is the intercept

### Step 3: Simulate response y, true model b, and the correct non-0 positions of b

In [6]:
y, true_b, correct_position = simulate_random_response(x, xbm, k, d, l)

([-2.02168310584216, -2.562563581153803, 1.2438984775887258, 0.30434816265962317, 1.7043321478245017, -2.7754953149035013, -0.9486637262536461, 0.1660538100586053, 1.5880052223831773, 1.033229535854426  …  -2.1664956144610645, 7.975518350862309, 0.32430592236910805, 1.6057303922701498, 1.5909304105723248, -2.503956200147935, -3.4652255843587714, -0.34640336293302176, 1.0706692918688652, 0.29268585188820095], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [2384, 3352, 4093, 5413, 5455, 6729, 7403, 8753, 9089, 9132])

### Step 4: Run IHT

In [7]:
result = L0_reg(x, xbm, z, y, 1, k, d(), l)


IHT estimated 10 nonzero SNP predictors and 0 non-genetic predictors.

Compute time (sec):     0.6195089817047119
Final loglikelihood:    -1407.2533232402275
Iterations:             12
Max number of groups:   1
Max predictors/group:   10

Selected genetic predictors:
10×2 DataFrame
│ Row │ Position │ Estimated_β │
│     │ Int64    │ Float64     │
├─────┼──────────┼─────────────┤
│ 1   │ 2384     │ -1.26014    │
│ 2   │ 3352     │ -0.26742    │
│ 3   │ 3353     │ 0.141208    │
│ 4   │ 4093     │ 0.289956    │
│ 5   │ 5413     │ 0.366689    │
│ 6   │ 5609     │ -0.137181   │
│ 7   │ 7403     │ -0.308255   │
│ 8   │ 8753     │ 0.332881    │
│ 9   │ 9089     │ 0.964598    │
│ 10  │ 9132     │ -0.509461   │

Selected nongenetic predictors:
0×2 DataFrame


### Step 5: Check results

IHT found 8/10 predictors in this example. The 2 that was not found had a relatively small effect size, and as far as IHT can tell, they are indistinguishable from noise. 

In [8]:
compare_model = DataFrame(
    true_β      = true_b[correct_position], 
    estimated_β = result.beta[correct_position])
@show compare_model
println("Total iteration number was " * string(result.iter))
println("Total time was " * string(result.time))
println("Total found predictors = " * string(length(findall(!iszero, result.beta[correct_position]))))

compare_model = 10×2 DataFrame
│ Row │ true_β    │ estimated_β │
│     │ Float64   │ Float64     │
├─────┼───────────┼─────────────┤
│ 1   │ -1.19376  │ -1.26014    │
│ 2   │ -0.230351 │ -0.26742    │
│ 3   │ 0.257181  │ 0.289956    │
│ 4   │ 0.344827  │ 0.366689    │
│ 5   │ 0.155484  │ 0.0         │
│ 6   │ -0.126114 │ 0.0         │
│ 7   │ -0.286079 │ -0.308255   │
│ 8   │ 0.327039  │ 0.332881    │
│ 9   │ 0.931375  │ 0.964598    │
│ 10  │ -0.496683 │ -0.509461   │
Total iteration number was 12
Total time was 0.6195089817047119
Total found predictors = 8


# Example 2: Case-control study controlling for sex

Case control studies are used when the phenotype in a binary count data. In this example, we simulate a case-control study, while controling for sex as a non-genetic covariate. 

The exact simulation code to generate the phenotype $y$ can be found at: https://github.com/biona001/MendelIHT.jl/blob/master/src/simulate_utilities.jl#L107

### Step 1: Simulat data with k true predictors, from distribution d and with link l.

In [9]:
n = 1000
p = 10000
k = 10
d = Bernoulli
l = canonicallink(d())

LogitLink()

### Step 2: construct snpmatrix, covariate files, and true model b

In [10]:
Random.seed!(1111) #set random seed 
x = simulate_random_snparray(n, p, undef)
xbm = SnpBitMatrix{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true); 
z = ones(n, 2) # first column is the intercept, second column the sex. 0 = male 1 = female
z[:, 2] .= rand(0:1, n);

### Step 3: simulate true models 

Here we used $k=8$ genetic predictors and 2 non-genetic predictors (intercept and sex). The simulation code in our package does not yet handle simulations with non-genetic predictors, so we must simulate these phenotypes manually. 

In [11]:
true_b = zeros(p) #genetic predictors
true_b[1:k-2] = randn(k-2)
shuffle!(true_b)
correct_position = findall(!iszero, true_b)
true_c = [1.0; 1.5] #non-genetic predictors: intercept & sex

2-element Array{Float64,1}:
 1.0
 1.5

### Step 4: simulate phenotype using genetic and nongenetic predictors

In [12]:
prob = GLM.linkinv.(l, xbm * true_b .+ z * true_c)
y = [rand(d(i)) for i in prob]
y = Float64.(y); #convert y to floating point numbers

### Step 5: run IHT

In [13]:
result = L0_reg(x, xbm, z, y, 1, k, d(), l)


IHT estimated 8 nonzero SNP predictors and 2 non-genetic predictors.

Compute time (sec):     2.31581711769104
Final loglikelihood:    -285.50519454191857
Iterations:             51
Max number of groups:   1
Max predictors/group:   10

Selected genetic predictors:
8×2 DataFrame
│ Row │ Position │ Estimated_β │
│     │ Int64    │ Float64     │
├─────┼──────────┼─────────────┤
│ 1   │ 1777     │ 0.342627    │
│ 2   │ 2960     │ -2.30193    │
│ 3   │ 4588     │ -0.677193   │
│ 4   │ 5075     │ 0.443162    │
│ 5   │ 5651     │ 0.378245    │
│ 6   │ 6086     │ 0.761058    │
│ 7   │ 6130     │ -0.9267     │
│ 8   │ 9283     │ -0.732392   │

Selected nongenetic predictors:
2×2 DataFrame
│ Row │ Position │ Estimated_β │
│     │ Int64    │ Float64     │
├─────┼──────────┼─────────────┤
│ 1   │ 1        │ 0.952232    │
│ 2   │ 2        │ 1.69576     │

### Step 6: check result

As we can see below, IHT finds 5/8 true genetic predictors and 2/2 true non-genetic predictors. Note that:

+ The coefficient estimates for found predictors are unbiased.
+ Larger effect sizes are easier to find.
+ Increasing number of samples will increase the accuracy of estimation. 

In [14]:
compare_model_genetic = DataFrame(
    true_β      = true_b[correct_position], 
    estimated_β = result.beta[correct_position])

compare_model_nongenetic = DataFrame(
    true_c      = true_c[1:2], 
    estimated_c = result.c[1:2])

@show compare_model_genetic
println("\n")
@show compare_model_nongenetic

compare_model_genetic = 8×2 DataFrame
│ Row │ true_β    │ estimated_β │
│     │ Float64   │ Float64     │
├─────┼───────────┼─────────────┤
│ 1   │ -2.22637  │ -2.30193    │
│ 2   │ 0.0646127 │ 0.0         │
│ 3   │ -0.63696  │ -0.677193   │
│ 4   │ 1.08631   │ 0.761058    │
│ 5   │ -0.930103 │ -0.9267     │
│ 6   │ -0.283783 │ 0.0         │
│ 7   │ -0.206074 │ 0.0         │
│ 8   │ -0.553461 │ -0.732392   │


compare_model_nongenetic = 2×2 DataFrame
│ Row │ true_c  │ estimated_c │
│     │ Float64 │ Float64     │
├─────┼─────────┼─────────────┤
│ 1   │ 1.0     │ 0.952232    │
│ 2   │ 1.5     │ 1.69576     │


,true_c,estimated_c
,Float64,Float64
1,1.0,0.952232
2,1.5,1.69576


# Example 3: Cross Validation with Poisson using debiasing

In this example, we investiate IHT's cross validation routines using as many CPU cores as possible. We use Poisson regression as an example. The current machine (4 cores avaialble) info is listed in the beginning of this tutorial. We also turned on debiasing just to show that this functionality work.

### Step 1: Verify we can multiple workers involved. 

Workers were added in the first example with the Distributed.jl package. If `nprocs()` return 1, restart the notebook and add workers before loading packages. 

In [15]:
nprocs()

5

### Step 2: simulat data with k true predictors, from distribution d and with link l.

Here we chose a larger sample size to have better accuracy.

In [16]:
n = 2000
p = 20000
k = 10
d = Poisson
l = canonicallink(d())

LogLink()

### Step 3: construct snpmatrix, covariate files, and true model b

In [17]:
Random.seed!(1111) #set random seed
x = simulate_random_snparray(n, p, undef)
xbm = SnpBitMatrix{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true); 
z = ones(n, 1); # the intercept

### Step 4: simulate response, true model b, and the correct non-0 positions of b

In [18]:
y, true_b, correct_position = simulate_random_response(x, xbm, k, d, l)

([0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 2.0, 2.0, 1.0, 5.0  …  7.0, 1.0, 2.0, 0.0, 0.0, 6.0, 3.0, 0.0, 6.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [17, 1023, 1411, 7670, 9983, 10963, 14145, 16020, 16260, 19148])

### Step 5: specify path and folds

Here `path` are all the model sizes you wish to test and `folds` indicates how to partition the samples into disjoint groups. It is important we partition the training/testing data randomly as opposed to chunck by chunck to avoid nasty things like sampling biases. Below we tested $k = 1, 2, ..., 20$ across 3 fold. This is equivalent to running IHT across 60 different models, and hence, is ideal for parallel computing. 

In [19]:
path = collect(1:20)
num_folds = 3
folds = rand(1:num_folds, size(x, 1));

### Step 6: Run IHT's cross validation routine

This returns a vector of deviance residuals, which is a generalization of the mean squared error. 

**Warning:** This step will generate intermediate files with `.bed` endings. These are necessary auxiliary files that will be automatically removed when cross validation completes. **Removing these files before the algorithm terminate will lead to bad errors.**

In [20]:
drs = cv_iht(d(), l, x, z, y, 1, path, num_folds, folds=folds, debias=true, parallel=true)



Crossvalidation Results:
	k	MSE
	1	1982.22839344047
	2	1312.6742338565018
	3	1048.9517275893654
	4	803.6204585340461
	5	727.9419536010191
	6	695.1505280021781
	7	682.8454101672021
	8	691.220415496229
	9	696.0332745251038
	10	700.6136354518109
	11	705.0504280518771
	12	717.0023707585339
	13	719.5101480774265
	14	726.6852303363696
	15	738.6231749358319
	16	740.5762703087061
	17	745.5001885112526
	18	754.1900731242154
	19	746.1113648837154
	20	768.5982812787013


20-element Array{Float64,1}:
 1982.22839344047  
 1312.6742338565018
 1048.9517275893654
  803.6204585340461
  727.9419536010191
  695.1505280021781
  682.8454101672021
  691.220415496229 
  696.0332745251038
  700.6136354518109
  705.0504280518771
  717.0023707585339
  719.5101480774265
  726.6852303363696
  738.6231749358319
  740.5762703087061
  745.5001885112526
  754.1900731242154
  746.1113648837154
  768.5982812787013

### Step 7: Run full model on the best estimated model size 

According to our cross validation result, the best model size that minimizes deviance residuals (i.e. MSE on the q-th subset of samples) is attained at $k = 10$. That is, cross validation detected that we need 10 SNPs to achieve the best model size. Using this information, one can re-run the IHT code to obtain the estimated model.

In [21]:
k_est = argmin(drs)
@show k_est
result = L0_reg(x, xbm, z, y, 1, k_est, d(), l, debias=true)

k_est = 7



IHT estimated 7 nonzero SNP predictors and 0 non-genetic predictors.

Compute time (sec):     1.7413861751556396
Final loglikelihood:    -2698.296299880555
Iterations:             8
Max number of groups:   1
Max predictors/group:   7

Selected genetic predictors:
7×2 DataFrame
│ Row │ Position │ Estimated_β │
│     │ Int64    │ Float64     │
├─────┼──────────┼─────────────┤
│ 1   │ 17       │ -0.107678   │
│ 2   │ 1023     │ -0.850201   │
│ 3   │ 1411     │ -0.492739   │
│ 4   │ 7670     │ -0.16534    │
│ 5   │ 14145    │ 0.355557    │
│ 6   │ 16020    │ -0.578028   │
│ 7   │ 19148    │ -0.227223   │

Selected nongenetic predictors:
0×2 DataFrame


### Step 8: Check final model against simulation

In [22]:
compare_model = DataFrame(
    true_β      = true_b[correct_position], 
    estimated_β = result.beta[correct_position])
@show compare_model
println("Total iteration number was " * string(result.iter))
println("Total time was " * string(result.time))
println("Total found predictors = " * string(length(findall(!iszero, result.beta[correct_position]))))

compare_model = 10×2 DataFrame
│ Row │ true_β     │ estimated_β │
│     │ Float64    │ Float64     │
├─────┼────────────┼─────────────┤
│ 1   │ -0.106645  │ -0.107678   │
│ 2   │ -0.839519  │ -0.850201   │
│ 3   │ -0.507777  │ -0.492739   │
│ 4   │ -0.167797  │ -0.16534    │
│ 5   │ -0.027083  │ 0.0         │
│ 6   │ -0.0120868 │ 0.0         │
│ 7   │ 0.374946   │ 0.355557    │
│ 8   │ -0.572632  │ -0.578028   │
│ 9   │ 0.0437437  │ 0.0         │
│ 10  │ -0.240008  │ -0.227223   │
Total iteration number was 8
Total time was 1.7413861751556396
Total found predictors = 7


# Conclusion

This notebook demonstrated some of the basic features of IHT. It is important to note that in the real world, the effect sizes of genetic predictors are expected to be small. Thus to detecting them would require a reasonably large sample size (say $n$ in the thousands). Fortunately, this is common place nowadays. 


# Extra features 

Due to limited space, we obmited illustrating some functionalities that have already been implemented, listed below:

+ Negative binomial, gamma, inverse gaussian, and binomial regressions
+ Use of non-canonical link functions 
+ Initializing IHT at a good starting point (setting init=true)
+ Doubly sparse projection (requires group information)
+ Weighted projections (requires weight information)

Interested users can visit [our code to reproduce certain figures of our paper](https://github.com/OpenMendel/MendelIHT.jl/tree/master/figures) on our github. 